# Physics in MiniCombust

Modelling the physics of combustion results in a system of interdependent equations representing chemistry, fluid dynamics (turbulent flow), thermodynamics, heat transfer, mass transfer and two-phase flow. These equations are non-linear PDEs, which we discretise on a computational grid and then solve the system of PDEs for each timestep of a simulation.

We present a very summarised overview of the equations MiniCombust tries to solve in this document, but it's worth a longer look in the Resources. We can particularly recommend Chapter 3 in 
the text [Computational Fluid Dynamics in Industrial Combustion, ed. Baukal, Gershtein and Li](./Resources.ipynb/#computational-fluid-dynamics-in-industrial-combustion).


## Transport Equations
The equations describing combustion (heat transfer, fluid motion etc.) are [_scalar transport equations ("continuiuty equation")_](https://en.wikipedia.org/wiki/Continuity_equation), which describe how some scalar quantity (let's call in $\phi$) is transported in a flow field. For a given [control volume](https://en.wikipedia.org/wiki/Control_volume), a scalar transport equation hasthe general form:

$$
\frac{\partial}{\partial t}(\rho \phi) + \mathrm{div}(\rho \mathbf{V} \phi) = \mathrm{div}(\Gamma_\phi\ \mathrm{grad}\ \phi) + q_\phi 
$$

This is the form given in the [Dolfyn](Dolfyn.ipynb) developer guide. Alternatively, using the nabla notation preferred in [Computational Fluid Dynamics in Industrial Combustion](#computational-fluid-dynamics-in-industrial-combustion)):

$$
\frac{\partial}{\partial t}(\rho \phi) + \nabla \cdot(\rho \mathbf{V} \phi) = \nabla \cdot(\Gamma_\phi \nabla \phi) + q_\phi 
$$

The scalar transport equation states that how $\phi$ varies in time is a balance of the convection through, and diffusion in and out of, the control volume, as well as the effect of a source or sink.

The four terms from left to right are known as the
* Transient term
* Convection term
* Diffusion term
* Source term

Each PDE representing some physical system has some or all of these terms, and we take different approaches to discretising each of the terms on an unstructured grid. The discretisation choices for MiniCombust are covered in the [Discretisations](#discretisation) documentation.

MiniCombust uses the Reynolds-averaged form of the coverning equations (see chapter 3 of [Computational Fluid Dynamics in Industrial Combustion](./Resources#computational-fluid-dynamics-in-industrial-combustion).


### 



## Fluid dynamics and transport
MiniCombust solves the _incompressible_  Navier-Stokes equations for low Reynolds numbers. We use the time-averaged form of these equations (RANS).
This give us two partial differential equations (Conservation of Mass, Conservation of Momentum). We also add a conservation equation for Energy.

To solve these, we will discretise them on an unstructured grid, and use a form of the SIMPLE algorithm, but first we describe the physics here:


## Conservation of mass (continuity equation)
Reynolds-averaged mixture continuity

$$
\frac{\partial}{\partial t}(\rho) + \nabla\cdot(\rho\mathbf{V}) = S_m
$$

* $t$ time
* $\rho$ Reynolds-averaged mixture density
* $\mathbf{V}$ Reynolds-averaged velocity vector
* $S_m$ external mass sources


## Conservation of momentum
Reynolds-averaged gas-phase momentum

$$
\frac{\partial}{\partial t}(\rho\mathbf{V}) + \nabla\cdot(\rho\mathbf{VV}) = \nabla\cdot((\mu + \mu_t)\nabla\mathbf{V}) + \mathbf{F}
$$

* $\mu_t$ turbulent viscosity (from turbulence model)
* $\mathbf{F}$ stress term and other momentum sources (e.g. drag)

Together, the Momentum, and Continuity equations frm the Navier-Stokes equations?

## Conservation of Energy
Equation:

$$
\frac{\partial}{\partial t}(\rho E) + \nabla \cdot(\rho \mathbf{V} E) = \nabla \cdot
((k + k_t)\nabla T) + \nabla \cdot (\tau \cdot \mathbf{V}) - \nabla \cdot (p \mathbf{V}) + S_r + S_h
$$

See [the equation for $S_r$ below](#radiative-heat-transfer).

* $k$ thermal conductivity
* $k_t$ turbulent thermal conductivity (from turbulence model)
* $\tau$ stress tensor
* $p$ pressure
* $E$ total energy
* $e$ internal energy per unit mass
* $S_r$ radiative heat soure
* $S_h$ reaction source terms and other heat sources


## Turbulence model
MiniCombust only supports the [$k-\epsilon$ turbulence model](https://en.wikipedia.org/wiki/K-epsilon_turbulence_model), which adds two transport equations to the set of PDEs:

### Turbulent kinetic energy ($k$)

$$
\frac{\partial}{\partial t}(\rho k) + \nabla \cdot (\rho \mathbf{V} k) = \nabla \cdot \left( \frac{(\mu + \mu_t)}{\sigma_k} \nabla k \right) + G_k - \rho \epsilon
$$

### Dissipation ($\epsilon$)

$$
\frac{\partial}{\partial t}(\rho \epsilon) + \nabla \cdot (\rho \mathbf{V} \epsilon) = \nabla \cdot \left( \frac{(\mu + \mu_t)}{\sigma_\epsilon} \nabla \epsilon \right) + C_{1 \epsilon}\frac{\epsilon}{k}G_{k} - C_{2_\epsilon}\rho\frac{\epsilon^2}{k}
$$

* $G_k$ is the turbulence production term
* $\sigma_k$ and $\sigma_\epsilon$ are the turbulence Prandtl numbers
* $C_{1 \epsilon}$ and $C_{2_\epsilon}$ are model constants

<a id="radiative-heat-transfer"></a>
## Radiative (heat) tranfer 

Radiative transfer equation, no radiation-turbulence interaction

$$
\nabla \cdot (I(\mathbf{s})\mathbf{s}) = -(\kappa + \sigma_s)I(\mathbf{s}) + B(\mathbf{s})
$$
, with

$$
B(\mathbf{s}) = \kappa I_B + \frac{\sigma_s}{4\pi}\int_{4\pi}I(\mathbf{s}')\Phi(\mathbf{s}', \mathbf{s})d\Omega'
$$ 
,and the term in the Energy Equation above

$$
S_r = \kappa\int_{4 \pi}[I(\mathbf{s} - I_B)]d\Omega
$$

* $I(\mathbf{s})$ radiant intensity in the direction $\mathbf{s}$
* $\kappa$, absorption coefficient
* $\sigma_s$, scattering coefficient
* $I_B$ blackbody intensity
* $\Phi$, scattering phase function

## Species transport
The Reynolds-averaged conservation equation for mass fraction of a species:

$$
\frac{\partial}{\partial t}(\rho m_1) + \nabla \cdot (\rho \mathbf{V} m_l) = \nabla \cdot \left( (\rho D + \frac{\mu_t}{\sigma_m})\nabla  m_l \right) + R_l
$$

* for a mass fraction $m_l$ f species $l$
* $D$, the diffusion coefficient for species $l$
* $\sigma_m$, turbulent Schmidt number
* $R_l$, Reaction rate in the eddy breakup model (minimum [Arrhenius rate](https://en.wikipedia.org/wiki/Arrhenius_equation).)

The only fuel modelled in MiniCombust is [methane](https://en.wikipedia.org/wiki/Methane#Combustion).

### Mixture fraction 
To model the mixture fraction, we use the PDF approach described in Section 3.2.7 of [CFDInIndustrialCombustion]. We use a model called FGM (Flamelet Generated Model), which adds
 two PDEs to the system, and then we interpolate the thermomechanical state of cells in the geometry using pre-computed tables for a Methane/Air mix. Grimech 3.0?

### Eddy breakup 
For turbulent flows, we use an [eddy disspation model](https://en.wikipedia.org/wiki/Methane#Combustion), the 2-step Westbrook-Dryer model for methane
TODO isn't this for laminar flames? What about Magnussen and Hjertage


Combustion Chemistry
## Emissions
* NOx model
* Soot model

## Dispersed phase model (particles)
MiniCombust uses a mixed Eulerian-Lagrangian (particle & flow simulation) approach, and tracks fuel particles injected at certain points in the geometry, at various times an rates in the simulation.

The mass, momentum, energy and composition (chemistry) of a particle is calculated along its trajectory.

We need to track the mass, energy and composition of the particles. 

We need to calculate energy transfer between particle and gas phase and account for vaporisation (spray).


In MiniCombust, the Spray and Evaporation models are incorporated into the Lagrangian (particle) system, and manifest as extra source terms in the
Mass Conservation and Mixture Fraction equations. For more details, see [Particles](Particles.ipynb)

Spherical droplets
Parcel approach?

## Atomisation (Spray)
## Spray breakup
## Evaporation
MiniCombust uses Chin and Lefebvre's evaporation model. Each particle loses mass with time according to

$$
\frac{dm_d}{dt} = 2\pi D_d(\frac{\lambda}{C_p})_g(1+B_m) 
$$
* $\lambda$ mean thermal conductivity
* $C_p$ specific heat of the fast
* B_m the mass transfer number defined by $B_m = \frac{{Y_F}_s}{1 - {Y_F}_s} $
* ${Y_F}_S$ is the mass fraction of fuel vapour at the droplet surface, given by

$$
{Y_F}_S = \left( 1 + \left( \frac{p}{{p_F}_S} - 1 \right)\frac{M_A}{M_F} \right)^{-1}
$$

* $p$ air pressure
* ${p_F}_S$ fuel vapour pressure ([Clausius-Clapeyron relation](https://en.wikipedia.org/wiki/Clausius%E2%80%93Clapeyron_relation))
* $M_A$ and $M_F$, molecular weight of air and the fuel

In addition to mass, heat is exchanged. Fuel droplets are modelled to have constant temperature equal to their surface temperature. As a droplet's surface changes, heat is exchanged (from air to fuel and heat absorbed through evaporation).


$$
\frac{dT_d}{dt} = \frac{Q - Qc}{C_{p,f}m_d}
$$
<a id="eqn-particle-temp-change"></a>




## Spray injection



<a id="discretisation"></a>
# Discretisation
MiniCFD uses the cell-centered Finite Volume Method for unstructured grids.

The discretisation of each of the equations in the above section involves choices for how to distretise the convective, diffusive and source terms on the unstructured grid.
We use colocated points at the cell centres, and have to correct for the complex geometry by performing interpolations.

The approach to discretising the Navier-Stokes equations is described in detail in [Peric](./Resources.ipynb#peric) (esp. Chapters 7 and 8, with 8.6 especially helpful).
In general, we follow the choices in Dolfyn exactly.

Second-order central differencing (linear interpolation)
Over-relaxed correction of the diffusive term ( https://www.cfd-online.com/Wiki/Discretization_of_the_diffusion_term)


# Also see
* https://www.thermopedia.com/content/643/
* https://www.cfd-online.com/Wiki/Discretization_of_the_diffusion_term